In [31]:
import os
import json
import openai
import guidance
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()

True

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
with open("./data/2023_11_KICE.json", "rb") as f:
    data = json.load(f)

In [6]:
total_score = 0
for pa in data:
    for problem in pa["problems"]:
        total_score += problem["score"]
assert (total_score == 100)

In [7]:
for pa in data:
    for problem in pa["problems"]:
        print(
            f'질문 : {problem["question"]}\n1번 : {problem["choices"][0]}\n2번 : {problem["choices"][1]}\n3번 : {problem["choices"][2]}\n4번 : {problem["choices"][3]}\n5번 : {problem["choices"][4]}\n정답 : {problem["answer"]}\n점수 : {problem["score"]}\n\n')

질문 : 윗글의 내용과 일치하지 않는 것은?
1번 : 같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.
2번 : 다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다
3번 : 독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.
4번 : 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
5번 : 독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다
정답 : 4
점수 : 2


질문 : 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
1번 : ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.
2번 : ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.
3번 : ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
4번 : ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다.
5번 : ⓓ에는 ‘독자 자신에게서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
정답 : 5
점수 : 3


질문 : 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
1번 : 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.
2번 : 독서 모임에서 서로 다른 관점을 확인하고 자신의 관점을 조정하는 과정에서 경험할 수 있겠군.
3번 : 개인적으로 형성한 의미를, 독서 동아리를 통해 심화하는 과정에서 경험할 수 있겠군.
4번 : 자신의 독서 경험을 담은 콘텐츠를 생산하고 공유하는 과정에서 경험할 수 있겠군.
5번 : 오프라인뿐 아니라 온라인 공간에서 해석을 나누는 과정에서도 경험할 수 있겠군.
정답 : 1
점수 : 2


질문 : (가)와 (나)에 대한

# Test

In [10]:
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [29]:
def get_answer(paragraph, question, choices, question_plus=""):
    system_prompt = """
        국어 시험 문제를 푸는 똑똑한 학생으로써 다음 문제의 답을 구하세요.
        지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    """
    user_prompt = f"""
        지문 :
        {paragraph}
    """
    if question_plus:
        user_prompt += f"""
            보기 :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요. 정답 :
    """

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }])
    return completion.choices[0].message.content

In [33]:
answers = list()
answer_num = 1
for pa in data:
    for problem in pa["problems"]:
        if "question_plus" in list(problem.keys()):
            question_plus_text = problem["question_plus"]
        else:
            question_plus_text = ""
        out = get_answer(paragraph=pa["paragraph"],
                         question=problem["question"],
                         choices=problem["choices"],
                         question_plus=question_plus_text)
        print(f'{answer_num}번 문제 : 답 : {out}')
        answer_num += 1
        answers.append(out)
        time.sleep(10)

1번 문제 : 답 : 4번
1번 문제 : 답 : 5번
1번 문제 : 답 : 1번
1번 문제 : 답 : 4번
1번 문제 : 답 : 5번
1번 문제 : 답 : 3번
1번 문제 : 답 : 2번
1번 문제 : 답 : 5번
1번 문제 : 답 : 2번
1번 문제 : 답 : 4번
1번 문제 : 답 : 5번
1번 문제 : 답 : 2번
1번 문제 : 답 : 5번
1번 문제 : 답 : 3번
1번 문제 : 답 : 3번
1번 문제 : 답 : 4번
1번 문제 : 답 : 1번
1번 문제 : 답 : 4번
1번 문제 : 답 : 3번
1번 문제 : 답 : 3번
1번 문제 : 답 : 4번
1번 문제 : 답 : 1번
1번 문제 : 답 : 5번
1번 문제 : 답 : 3번
1번 문제 : 답 : 3번
1번 문제 : 답 : 4번
1번 문제 : 답 : 1번
1번 문제 : 답 : 5번
1번 문제 : 답 : 5번
1번 문제 : 답 : 2번
1번 문제 : 답 : 5번
1번 문제 : 답 : 4번
1번 문제 : 답 : 2번
1번 문제 : 답 : 3번
1번 문제 : 답 : 1번
1번 문제 : 답 : 5번
1번 문제 : 답 : 3번
1번 문제 : 답 : 4번
1번 문제 : 답 : 4번
1번 문제 : 답 : 1번
1번 문제 : 답 : 4번
1번 문제 : 답 : 4번
1번 문제 : 답 : 5번
1번 문제 : 답 : 2번
1번 문제 : 답 : 1번


In [23]:
solutions = list()
for pa in data:
    for problem in pa["problems"]:
        solutions.append(problem["answer"])

In [25]:
scores = list()
for pa in data:
    for problem in pa["problems"]:
        scores.append(problem["score"])

In [34]:
f = open("./result/2023_11_KICE-gpt4_baseline.txt", 'w')
for i, item in enumerate(answers):
    txt = f'{i+1}번 문제 : {item}\n정답 : {solutions[i]}\n배점 : {scores[i]}\n----------------------------\n'
    print(txt)
    f.write(txt)
f.close()

1번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

2번 문제 : 5번
정답 : 5
배점 : 3
----------------------------

3번 문제 : 1번
정답 : 1
배점 : 2
----------------------------

4번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

5번 문제 : 5번
정답 : 5
배점 : 2
----------------------------

6번 문제 : 3번
정답 : 3
배점 : 2
----------------------------

7번 문제 : 2번
정답 : 2
배점 : 2
----------------------------

8번 문제 : 5번
정답 : 5
배점 : 3
----------------------------

9번 문제 : 2번
정답 : 2
배점 : 2
----------------------------

10번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

11번 문제 : 5번
정답 : 5
배점 : 2
----------------------------

12번 문제 : 2번
정답 : 2
배점 : 3
----------------------------

13번 문제 : 5번
정답 : 5
배점 : 2
----------------------------

14번 문제 : 3번
정답 : 3
배점 : 2
----------------------------

15번 문제 : 3번
정답 : 4
배점 : 2
----------------------------

16번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

17번 문제 : 1번
정답 : 1
배점 : 3
----------------------------

18번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

1